# Определение места бурения новых скважин

#### Описание работы :

Необходимо постройть модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. А также проанализировать возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

#### Описание данных :

* `id` — уникальный идентификатор скважины;
* `f0, f1, f2` — три признака точек;
* `product` — объём запасов в скважине (тыс. баррелей).

#### План работы над проектом :

   1. [Подготовка данных:](#Step_1)
       * [Обзор и загрузка данных](#Step_1_1)
       * [Обработка дубликатов](#Step_1_2)
       * [Вывод](#Step_1_3)
   2. [Обучение и проверка модели для каждого региона](#Step_2)
       * [Разделение данных на обучающую и валидационную выборки](#Step_2_1)
       * [Обучение модели и прогноз для валидационной выборки](#Step_2_2)
       * [Вывод](#Step_2_3)
   3. [Подготовка к расчёту прибыли](#Step_3)
       * [Ключевые значения для расчётов](#Step_3_1)
       * [Рассчёт объёма сырья для безубыточной разработки новой скважины. Сравнение полученных объёмов сырья со средним запасом в каждом регионе ](#Step_3_2)
       * [Вывод](#Step_3_3)
   4. [Расчёт прибыли по выбранным скважинам и предсказаниям модели](#Step_4)
   5. [Расчёт риска и прибыли для каждого региона](#Step_5)
   6. [Общий вывод](#Step_6)

# 1. Подготовка данных <a id="Step_1"></a>

### 1.1 Обзор и загрузка данных <a id="Step_1_1"></a>

Загрузим необходимые для работы библиотеки.

In [2]:
import pandas as pd

import numpy as np

# для разделения данных на выборки
from sklearn.model_selection import train_test_split

# для использования алгоритма линейной регрессии
from sklearn.linear_model import LinearRegression

# для расчёта метрики MSE
from sklearn.metrics import mean_squared_error

# для расчёта доверительного интервала
from scipy import stats as st

# для использования масштабирования
from sklearn.preprocessing import StandardScaler 

Откроем файлы с данными и изучим информацию для каждого из регионов.

*Регион А* 

In [3]:
df_a = pd.read_csv('/datasets/geo_data_0.csv', sep=',')

In [4]:
display(df_a)
df_a.info()
df_a.describe()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
...,...,...,...,...,...
99995,DLsed,0.971957,0.370953,6.075346,110.744026
99996,QKivN,1.392429,-0.382606,1.273912,122.346843
99997,3rnvd,1.029585,0.018787,-1.348308,64.375443
99998,7kl59,0.998163,-0.528582,1.583869,74.040764


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


*Регион B* 

In [5]:
df_b = pd.read_csv('/datasets/geo_data_1.csv', sep=',')

In [6]:
display(df_b)
df_b.info()
df_b.describe()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
...,...,...,...,...,...
99995,QywKC,9.535637,-6.878139,1.998296,53.906522
99996,ptvty,-10.160631,-12.558096,5.005581,137.945408
99997,09gWa,-7.378891,-3.084104,4.998651,137.945408
99998,rqwUm,0.665714,-6.152593,1.000146,30.132364


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


*Регион C*

In [7]:
df_c = pd.read_csv('/datasets/geo_data_2.csv', sep=',')

In [8]:
display(df_c)
df_c.info()
df_c.describe()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
...,...,...,...,...,...
99995,4GxBu,-1.777037,1.125220,6.263374,172.327046
99996,YKFjq,-1.261523,-0.894828,2.524545,138.748846
99997,tKPY3,-1.199934,-2.957637,5.219411,157.080080
99998,nmxp2,-2.419896,2.417221,-5.548444,51.795253


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


Для каждого из регионов смысловая нагрузка столбцов, а также их количество, полностью совпадают как между собой, так и с описанием данных, сделанным в начале проекта. Пропусков нет, названия столбцов в нижнем регистре, к типу данных также вопросов нет.

### 1.2 Обработка дубликатов  <a id="Step_1_2"></a>

Проверим данные для каждого региона на наличие полных дубликатов.

Здесь и далее перед работой с данными того или иного региона будет ставиться только соответствующий символ (A,B,C).

A

In [9]:
print(df_a.duplicated().sum()) 

0


B

In [10]:
print(df_b.duplicated().sum()) 

0


C

In [11]:
print(df_c.duplicated().sum()) 

0


Полные дубликаты отсутствуют. Проверим дубликаты по уникальным идентификатором `id` скважин.

A

In [12]:
print(df_a['id'].duplicated().sum()) 

10


В целом, речь о 20 объектах из 100000, поэтому простое исключение этих данных без дополнительного анализа сколь-нибудь существенной роли для нас не сыграет.

Для реальных, а не синтетических данных, причиной подобного рода дубликатов мог послужить как сбой в работе автоматики, так и человеческий фактор при указании названия той или иной скважины.

После удаления дубликатов, столбец с уникальным идентификатором скважин `id`, по сути, становится для нас избыточной информацией, подлежащей удалению, так как его роль, с таким же успехом, будет выполнять индексация в наших фреймах.  

Аналогичным образом поступим и для других регионов.

In [13]:
df_a_dub = df_a['id'].value_counts().head(10)

In [14]:
df_a = df_a.query('id not in @df_a_dub.index')

In [15]:
df_a.drop(['id'], axis = 'columns', inplace = True)

B

In [16]:
print(df_b['id'].duplicated().sum()) 

4


In [17]:
df_b_dub = df_b['id'].value_counts().head(4)

In [18]:
df_b = df_b.query('id not in @df_b_dub.index')

In [19]:
df_b.drop(['id'], axis = 'columns', inplace = True)

C

In [20]:
print(df_c['id'].duplicated().sum()) 

4


In [21]:
df_c_dub = df_c['id'].value_counts().head(4)

In [22]:
df_c = df_c.query('id not in @df_c_dub.index')

In [23]:
df_c.drop(['id'], axis = 'columns', inplace = True)

### 1.3 Вывод  <a id="Step_1_3"></a>

   1. *Данные и необходимые для работы библиотеки успешно загружены.*
   2. *Проведён первичный анализ. Обнаружены и удалены дубликаты, а также избыточная информация (признак `id`). Данные готовы к дальнейшему анализу.*

# 2. Обучение и проверка модели для каждого региона <a id="Step_2"></a>

### 2.1 Разделение данных на обучающую и валидационную выборки <a id="Step_2_1"></a>

Разделим исходные данные на обучающую и валидационную выборки в пропорции 75% / 25% соответственно, сразу выделив признаки и целовой признак, для каждого региона. Масштабируем признаки. Также сбросим индексы до нумерации по порядку в случае `target_valid` для корректности последующих расчётов.

А

In [24]:
features_a = df_a.drop(['product'], axis = 1) 
target_a = df_a['product']

In [25]:
features_a_train, features_a_valid, target_a_train, target_a_valid = train_test_split(
    features_a, target_a, test_size= 0.25, random_state=12345)

In [26]:
scaler = StandardScaler()
scaler.fit(features_a_train) 
features_a_train = scaler.transform(features_a_train)
features_a_valid= scaler.transform(features_a_valid) 

In [27]:
target_a_valid = target_a_valid.reset_index(drop = True)

B

In [28]:
features_b = df_b.drop(['product'], axis = 1) 
target_b = df_b['product']

In [29]:
features_b_train, features_b_valid, target_b_train, target_b_valid = train_test_split(
    features_b, target_b, test_size= 0.25, random_state=12345)

In [30]:
scaler = StandardScaler()
scaler.fit(features_b_train) 
features_b_train = scaler.transform(features_b_train)
features_b_valid = scaler.transform(features_b_valid) 

In [31]:
target_b_valid = target_b_valid.reset_index(drop = True)

C

In [32]:
features_c = df_c.drop(['product'], axis = 1) 
target_c = df_c['product']

In [33]:
features_c_train, features_c_valid, target_c_train, target_c_valid = train_test_split(
    features_c, target_c, test_size= 0.25, random_state=12345)

In [34]:
scaler = StandardScaler()
scaler.fit(features_c_train) 
features_c_train = scaler.transform(features_c_train)
features_c_valid = scaler.transform(features_c_valid) 

In [35]:
target_c_valid = target_c_valid.reset_index(drop = True)

### 2.2 Обучение модели и прогноз для валидационной выборки <a id="Step_2_2">

Обучим модель для каждого из регионов и сделаем прогноз на валидационной выборке, сразу преобразовав полученные результаты в `Series` для дальнейших расчётов. 

Также рассчитаем и выведем на экран средний запас предсказанного сырья и RMSE.

A

In [36]:
model = LinearRegression()
model.fit(features_a_train, target_a_train)
predictions_a_valid = pd.Series(model.predict(features_a_valid))

In [37]:
mean_a = round((predictions_a_valid.mean()),4)
rmse_a = round((mean_squared_error(target_a_valid, predictions_a_valid) ** 0.5), 4)

In [38]:
print('Средний запас предсказанного сырья для региона A:', mean_a)
print('RMSE для региона A:', rmse_a)

Средний запас предсказанного сырья для региона A: 92.4238
RMSE для региона A: 37.7169


B

In [39]:
model = LinearRegression()
model.fit(features_b_train, target_b_train)
predictions_b_valid = pd.Series(model.predict(features_b_valid))

In [40]:
mean_b = round((predictions_b_valid.mean()),4)
rmse_b = round((mean_squared_error(target_b_valid, predictions_b_valid) ** 0.5), 4)

In [41]:
print('Средний запас предсказанного сырья для региона B:', mean_b)
print('RMSE для региона B:', rmse_b)

Средний запас предсказанного сырья для региона B: 68.9831
RMSE для региона B: 0.8915


C

In [42]:
model = LinearRegression()
model.fit(features_c_train, target_c_train)
predictions_c_valid = pd.Series(model.predict(features_c_valid))

In [43]:
mean_c = round((predictions_c_valid.mean()),4)
rmse_c = round((mean_squared_error(target_c_valid, predictions_c_valid) ** 0.5), 4)

In [44]:
print('Средний запас предсказанного сырья для региона C:', mean_c)
print('RMSE для региона C:', rmse_c)

Средний запас предсказанного сырья для региона C: 95.1162
RMSE для региона C: 39.9755


Для наглядности, представим полученные результаты в табличном виде.

In [45]:
df_pivot_region = pd.DataFrame(
     {'region': ['A', 'B', 'C'], 
      'mean': [mean_a, mean_b, mean_c ],
      'rmse': [rmse_a, rmse_b, rmse_c ]})

### 2.3 Вывод  <a id="Step_2_3">

In [46]:
 display(df_pivot_region)

,region,mean,rmse
0,A,92.4238,37.7169
1,B,68.9831,0.8915
2,C,95.1162,39.9755


Мы построили модели на алгоритме линейной регрессии для каждого из регионов. Из полученных данных, приведённых в таблице выше, видно, что регион A и C существенно впереди по показателю среднего предсказанного сырья. Но с другой стороны, корень из среднеквадратичной ошибки, или, проще говоря, сама ошибка предсказания линейной регрессии, выраженная в тех же единицах, что и предсказываемая величина, у регионов A и C также очень высока, в то время, как у модели для региона B RMSE носит почти символический характер. 

Другими словами, если руководствоваться принципом оценки моделей по худшему результату, то мы в итоге на практике, в среднем, можем получить меньшие запасы для регионов A и C, чем для B.

На данном этапе решения поставленной задачи, на мой взгляд, более корректно было бы отдать предпочтение региону B, по которому нам удалось получить более устойчивый прогноз, который в свою очередь в реальных условиях может дать ещё и большие результаты, чем остальные два региона. А для бизнеса, чем точнее прогноз и меньше риски, тем, очевидно, лучше.

# 3. Подготовка к расчёту прибыли <a id="Step_3"></a>

### 3.1 Ключевые значения для расчётов  <a id="Step_3_1">

Сохраним ключевые значения для расчётов в переменных.

In [47]:
# бюджет на разработку скважин в регионе
BUDGET = 10000000000

# всего скважин для региона
N_ALL = 500

# лучшие скважины для разработки в регионе
N_BEST = 200

# доход с тысячи баррелей
INCOME = 450000

### 3.2 Рассчёт объёма сырья для безубыточной разработки новой скважины. Сравнение полученных объёмов сырья со средним запасом в каждом регионе <a id="Step_3_2">

Рассчитаем достаточный объёма сырья для безубыточной разработки новой скважины.

In [48]:
# BEP (англ. break-even point)
BEP = BUDGET / (INCOME * N_BEST)
print('BEP = ', BEP)

BEP =  111.11111111111111


Точку безубыточности получили, теперь сравним это значение со средними запасами сырья в регионе.

In [49]:
print('Среднее количество сырья в регионе A для одной скважины = ', round((df_a['product'].mean()),4))
print('Среднее количество сырья в регионе B для одной скважины = ', round((df_b['product'].mean()),4))
print('Среднее количество сырья в регионе C для одной скважины = ', round((df_c['product'].mean()),4))

Среднее количество сырья в регионе A для одной скважины =  92.4992
Среднее количество сырья в регионе B для одной скважины =  68.8241
Среднее количество сырья в регионе C для одной скважины =  94.9988


### 3.3 Вывод  <a id="Step_3_3">

Наша точка безубыточности для одной скважины составляет 111,12 тыс.баррелей. В тоже время средние запасы сырья для одной скважины в каждом из регионов заметно ниже этой цифры. Чтобы компании избежать потенциального убытка, очевидно, нам нужны лучшие 200 скважин из нашего набора, имеющие запасы выше рассчитаной точки безубыточности. 

# 4. Расчёт прибыли по выбранным скважинам и предсказаниям модели <a id="Step_4"></a>

Напишем функцию `revenue()` для расчёта прибыли по выбранным скважинам и предсказаниям модели. Функция будет иметь следующие аргументы: 
 - `target_valid` - целевое значение объёма сырья;
 - `predictions_valid` - предсказанные значения объёма сырья;
 - `count` - количество выбранных скважин.

In [50]:
def revenue(target_valid, predictions_valid, count):
    
    # упорядочим предсказанные значение по убыванию, чтобы далее выбрать лучшие скважины
    predictions_valid_sorted = predictions_valid.sort_values(ascending = False)
    
    # выберем лучшие скважины из целевых значений по индексам, полученным выше в результате упорядочивания
    # предсказаний
    selected = target_valid[predictions_valid_sorted.index][:count]
    
    # рассчитаем прибыль для полученного объёма сырья по выбранным скважинам
    return round((INCOME * selected.sum() / 1000000000 - 10),6)  

# 5. Расчёт риска и прибыли для каждого региона <a id="Step_5"></a>

Теперь напишем функцию позволяющую получить нам следующие результаты: 
 - распределение прибыли с помощью техники `Bootstrap`;
 - среднюю прибыль для полученного распределения; 
 - 95%-ый доверительный интервал (0.975 и 0.025 квантили);
 - 2.5% квантиль распределения, позволяющий оценить риск убытков для полученного распределения.
 
Рассматривать будем 500 скважин, а выберем 200 лучших.

In [72]:
def bootstrap(target_valid, predictions_valid):
    
    state = np.random.RandomState(12345)
    values = []

    for i in range(1000):
    
        target_a_valid_500 = target_valid.sample( n = N_ALL, replace = True
                                                 , random_state=state)
        predictions_a_valid_500 = predictions_valid[target_a_valid_500.index] 
        values.append(revenue(target_a_valid_500,predictions_a_valid_500, 200))

    values = pd.Series(values)

    mean = values.mean()
    risk = len(values[values < 0]) / len(values)
    q_2_5 = values.quantile(0.025)
    q_97_5 = values.quantile(0.975)
    
    print("Средняя выручка:{:.6f}".format(mean))
    print("Риск убытков: {:.2%}".format(risk))
    print("2.5%-квантиль:{:.6f}".format(q_2_5))
    print("97.5%-квантиль:{:.6f}".format(q_97_5))

Применим написанную фунцкию к каждому региону.

A

In [74]:
bootstrap(target_a_valid, predictions_a_valid)

Средняя выручка:0.459086
Риск убытков: 4.70%
2.5%-квантиль:-0.090234
97.5%-квантиль:0.987954


B

In [75]:
bootstrap(target_b_valid, predictions_b_valid)

Средняя выручка:0.535247
Риск убытков: 1.40%
2.5%-квантиль:0.086681
97.5%-квантиль:0.958082


C

In [76]:
bootstrap(target_c_valid, predictions_c_valid)

Средняя выручка:0.351244
Риск убытков: 11.40%
2.5%-квантиль:-0.174808
97.5%-квантиль:0.879066


Для большей наглядности представим полученные результаты в виде таблицы.

In [77]:
df_pivot_region_revenue = pd.DataFrame(
     {'region': ['A', 'B', 'C'], 
      'mean_revenue': [0.459086, 0.535247, 0.351244 ],
      'risk': ['4.70%', '1.40%', '11.40%'],
      'confidence_interval': [' -0.090234 - 0.987954'
                              , '0.086681 - 0.958082'
                              , ' -0.174808 - 0.879066' ]})

In [78]:
df_pivot_region_revenue

,region,mean_revenue,risk,confidence_interval
0,A,0.459086,4.70%,-0.090234 - 0.987954
1,B,0.535247,1.40%,0.086681 - 0.958082
2,C,0.351244,11.40%,-0.174808 - 0.879066


Очевидно, что безусловным лидером для начала разработки месторождения/ий является регион B.

Более подробно о причинах такого выбора и анализе поговорим в общем выводе. 

# 6. Общий вывод <a id="Step_6"></a>

На основании данных, полученных от добывающей компании, нам было необходимо спрогнозировать какой же из трёх регионов (A,B,C) является наиболее рентабельным для запуска разработки имеющихся там запасов. 

Мы загрузили данные и необходимые библиотеки для работы, изучили полученную информацию, провели минимально необходимую предобработку. Далее, воспользовавшись алгоритмом линейной регрессии, построили модель для каждого региона. Лучший результат относительно показателя RMSE был получен для региона B, что хорошо видно из приведённой ниже таблицы. То есть эта модель ошибалась существенно меньше, чем её конкуренты, построенные для остальных регионов. 

In [58]:
df_pivot_region

,region,mean,rmse
0,A,92.4238,37.7169
1,B,68.9831,0.8915
2,C,95.1162,39.9755


После этого, используя технику `Bootstrap`, мы произвели расчёт средней прибыли, рисков и 95%-ого доверительного интервала для 1000 результатов прибыли 200 лучших скважин (из 500 выбранных) для каждого из регионов. Результаты этих расчётов представленны ниже. 

In [79]:
df_pivot_region_revenue

,region,mean_revenue,risk,confidence_interval
0,A,0.459086,4.70%,-0.090234 - 0.987954
1,B,0.535247,1.40%,0.086681 - 0.958082
2,C,0.351244,11.40%,-0.174808 - 0.879066


Анализируя полученные метрики распределений для каждого из регионов, легко заметить, что результаты однозначно в пользу разработки региона B. Самая высокая средняя прибыль (0.535247 млрд), приемлимый уровень риска, когда вероятность убытков ниже 2,5% (только 1,4% для этого региона), чего не удалось получить для двух других регионов, а также высокий верхний порог для прибыли по верхней границе доверительного интервала. 

Другими словами, компании однозначно стоит начать с разработки именно этого региона.